## Model training for 5 percent data variation
1. Batch size = 64
2. Neurons = 128


In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
# from google.colab import drive
# drive.mount('/content/gdrive')
import os

In [14]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May6rd5percent_RLV_data 100 points2 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
input = output =df.values
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [15]:
def create_model2(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(7,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [16]:
i = 7;
model = create_model2(2**i)
# model.load_weights(f"/content/gdrive/My Drive/Last data /1percent model/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt")

In [17]:
model.predict(X_test)


array([[0.50000703, 0.49999532],
       [0.5000058 , 0.49999785],
       [0.50001115, 0.49999538],
       ...,
       [0.50000584, 0.4999982 ],
       [0.500006  , 0.49999794],
       [0.50001043, 0.49999237]], dtype=float32)

In [18]:
y_test

array([[0.06554793, 0.48950587],
       [0.77689939, 0.47137161],
       [0.13451512, 0.6880959 ],
       ...,
       [0.64493052, 0.48145433],
       [0.91585227, 0.47448883],
       [0.05393215, 0.6612795 ]])

In [19]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss = tf.keras.losses.mean_squared_error
def lr_sch3(epoch,lr):
    if (epoch >0) &(epoch<300):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>300):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
print("The Number of Hidden units used is: ",2**i)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch3,verbose = 1)
bs = 64;
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 20
checkpoint_path = f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/5 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')

The Number of Hidden units used is:  128


In [20]:
hist = model.fit(X_train,y_train,epochs = 1000,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/1000
2950/2950 [==============================] - 13s 4ms/step - loss: 0.0087 - accuracy: 0.8591 - val_loss: 0.0060 - val_accuracy: 0.8825 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/1000
2950/2950 [==============================] - 12s 4ms/step - loss: 0.0053 - accuracy: 0.8733 - val_loss: 0.0052 - val_accuracy: 0.8712 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/1000
2950/2950 [==============================] - 12s 4ms/step - loss: 0.0049 - accuracy: 0.8775 - val_loss: 0.0045 - val_accuracy: 0.8767 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/1000
2950/2950 [==============================] - 12s 4ms/step - loss: 0.0045 - accuracy: 0.8812 - val_loss: 0.0044 - val_accuracy: 0.8715 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/1000
2950/2950 [================

Epoch 36/1000
2950/2950 [==============================] - 12s 4ms/step - loss: 0.0021 - accuracy: 0.9269 - val_loss: 0.0026 - val_accuracy: 0.9091 - lr: 0.0010

Epoch 37: LearningRateScheduler setting learning rate to 0.001.
Epoch 37/1000
2950/2950 [==============================] - 12s 4ms/step - loss: 0.0021 - accuracy: 0.9280 - val_loss: 0.0023 - val_accuracy: 0.9195 - lr: 0.0010

Epoch 38: LearningRateScheduler setting learning rate to 0.001.
Epoch 38/1000
2950/2950 [==============================] - 13s 4ms/step - loss: 0.0020 - accuracy: 0.9301 - val_loss: 0.0032 - val_accuracy: 0.8992 - lr: 0.0010

Epoch 39: LearningRateScheduler setting learning rate to 0.001.
Epoch 39/1000
2950/2950 [==============================] - 13s 4ms/step - loss: 0.0020 - accuracy: 0.9292 - val_loss: 0.0018 - val_accuracy: 0.9293 - lr: 0.0010

Epoch 40: LearningRateScheduler setting learning rate to 0.001.
Epoch 40/1000
2935/2950 [============================>.] - ETA: 0s - loss: 0.0021 - accuracy: 0.


Epoch 102: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 102/1000
2950/2950 [==============================] - 13s 4ms/step - loss: 8.3464e-04 - accuracy: 0.9650 - val_loss: 5.7373e-04 - val_accuracy: 0.9742 - lr: 4.0660e-04

Epoch 103: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 103/1000
2950/2950 [==============================] - 13s 4ms/step - loss: 8.5473e-04 - accuracy: 0.9653 - val_loss: 9.9144e-04 - val_accuracy: 0.9508 - lr: 4.0660e-04

Epoch 104: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 104/1000
2950/2950 [==============================] - 13s 4ms/step - loss: 8.2594e-04 - accuracy: 0.9645 - val_loss: 8.4706e-04 - val_accuracy: 0.9640 - lr: 4.0660e-04

Epoch 105: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 105/1000
2950/2950 [==============================] - 12s 4ms/step - loss: 8.2697e-04 - accuracy: 0.9656 - val_loss: 0.0011 - val_accuracy: 0.9522 - lr: 4.0660e-04

Epoch 106: LearningRateSche

KeyboardInterrupt: 

model.save(f"/content/gdrive/My Drive/Last data /5percent model/best models/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt")

In [22]:
def lr_sch160(epoch,lr):
    epoch = epoch+160
    if (epoch >0) &(epoch<150):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>150):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
lr_scheduler2 = tf.keras.callbacks.LearningRateScheduler(lr_sch160,verbose = 1)
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))

In [23]:
checkpoint_path = f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/5 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt"
model.load_weights(checkpoint_path)

In [ ]:
hist = model.fit(X_train,y_train,epochs = 600,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler2,cp_callback2],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.000220405.
Epoch 1/600
2950/2950 [==============================] - 13s 4ms/step - loss: 5.4931e-04 - accuracy: 0.9748 - val_loss: 4.7260e-04 - val_accuracy: 0.9801 - lr: 2.2040e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.000220405.
Epoch 2/600
2950/2950 [==============================] - 12s 4ms/step - loss: 5.0078e-04 - accuracy: 0.9770 - val_loss: 3.8616e-04 - val_accuracy: 0.9800 - lr: 2.2040e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.000220405.
Epoch 3/600
2950/2950 [==============================] - 12s 4ms/step - loss: 5.0977e-04 - accuracy: 0.9767 - val_loss: 6.9960e-04 - val_accuracy: 0.9596 - lr: 2.2040e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.000220405.
Epoch 4/600
2950/2950 [==============================] - 13s 4ms/step - loss: 5.1399e-04 - accuracy: 0.9769 - val_loss: 5.4020e-04 - val_accuracy: 0.9744 - lr: 2.2040e-04

Epoch 5: LearningRateScheduler sett

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2950/2950 [==============================] - 13s 4ms/step - loss: 4.2232e-04 - accuracy: 0.9802 - val_loss: 4.1046e-04 - val_accuracy: 0.9832 - lr: 1.5924e-04

Epoch 43: LearningRateScheduler setting learning rate to 0.000159242.
Epoch 43/600
2950/2950 [==============================] - 13s 4ms/step - loss: 4.2147e-04 - accuracy: 0.9803 - val_loss: 3.6745e-04 - val_accuracy: 0.9821 - lr: 1.5924e-04

Epoch 44: LearningRateScheduler setting learning rate to 0.000159242.
Epoch 44/600
2950/2950 [==============================] - 12s 4ms/step - loss: 4.1814e-04 - accuracy: 0.9801 - val_loss: 6.7283e-04 - val_accuracy: 0.9628 - lr: 1.5924e-04

Epoch 45: LearningRateScheduler setting learning rate to 0.000159242.
Epoch 45/600
2950/2950 [==============================] - 13s 4ms/step - loss: 4.1163e-04 - accuracy: 0.9806 - val_loss: 4.6713e-04 - val_accuracy: 0.9720 - lr: 1.5924e-04

Epoch 46: LearningRateScheduler setting learning rate to 0.000159242.
Epoch 46/600
2950/2950 [=================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2934/2950 [============================>.] - ETA: 0s - loss: 3.8149e-04 - accuracy: 0.9816
Epoch 80: saving model to /home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/5 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt
2950/2950 [==============================] - 13s 4ms/step - loss: 3.8132e-04 - accuracy: 0.9816 - val_loss: 3.2272e-04 - val_accuracy: 0.9858 - lr: 1.3536e-04

Epoch 81: LearningRateScheduler setting learning rate to 0.000115053.
Epoch 81/600
2950/2950 [==============================] - 12s 4ms/step - loss: 3.6728e-04 - accuracy: 0.9823 - val_loss: 2.8667e-04 - val_accuracy: 0.9869 - lr: 1.1505e-04

Epoch 82: LearningRateScheduler setting learning rate to 0.000115053.
Epoch 82/600
2950/2950 [==============================] - 13s 4ms/step - loss: 3.6302e-04 - accuracy: 0.9821 - val_loss: 3.2733e-04 - val_accuracy: 0.9849 - lr: 1.1505e-04

Epoch 83: LearningRateScheduler setting learning rate to 0.000115053.
Epoch 83/600
2950/2950 [

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2950/2950 [==============================] - 13s 4ms/step - loss: 1.9768e-04 - accuracy: 0.9902 - val_loss: 2.0844e-04 - val_accuracy: 0.9901 - lr: 1.3910e-05

Epoch 352: LearningRateScheduler setting learning rate to 1.391e-05.
Epoch 352/600
2950/2950 [==============================] - 13s 4ms/step - loss: 1.9797e-04 - accuracy: 0.9898 - val_loss: 2.3464e-04 - val_accuracy: 0.9877 - lr: 1.3910e-05

Epoch 353: LearningRateScheduler setting learning rate to 1.391e-05.
Epoch 353/600
2950/2950 [==============================] - 13s 4ms/step - loss: 1.9467e-04 - accuracy: 0.9900 - val_loss: 2.0864e-04 - val_accuracy: 0.9903 - lr: 1.3910e-05

Epoch 354: LearningRateScheduler setting learning rate to 1.391e-05.
Epoch 354/600
2950/2950 [==============================] - 12s 4ms/step - loss: 1.9522e-04 - accuracy: 0.9900 - val_loss: 2.9251e-04 - val_accuracy: 0.9851 - lr: 1.3910e-05

Epoch 355: LearningRateScheduler setting learning rate to 1.391e-05.
Epoch 355/600
2950/2950 [=================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2950/2950 [==============================] - 13s 4ms/step - loss: 1.8989e-04 - accuracy: 0.9905 - val_loss: 2.0407e-04 - val_accuracy: 0.9904 - lr: 1.0050e-05

Epoch 388: LearningRateScheduler setting learning rate to 1.005e-05.
Epoch 388/600
2950/2950 [==============================] - 12s 4ms/step - loss: 1.8845e-04 - accuracy: 0.9905 - val_loss: 2.0892e-04 - val_accuracy: 0.9892 - lr: 1.0050e-05

Epoch 389: LearningRateScheduler setting learning rate to 1.005e-05.
Epoch 389/600
2950/2950 [==============================] - 13s 4ms/step - loss: 1.9028e-04 - accuracy: 0.9903 - val_loss: 2.0656e-04 - val_accuracy: 0.9906 - lr: 1.0050e-05

Epoch 390: LearningRateScheduler setting learning rate to 1.005e-05.
Epoch 390/600
2950/2950 [==============================] - 13s 4ms/step - loss: 1.8823e-04 - accuracy: 0.9906 - val_loss: 2.1254e-04 - val_accuracy: 0.9903 - lr: 1.0050e-05

Epoch 391: LearningRateScheduler setting learning rate to 1.005e-05.
Epoch 391/600
2950/2950 [=================


Epoch 420: LearningRateScheduler setting learning rate to 8.543e-06.
Epoch 420/600
2917/2950 [============================>.] - ETA: 0s - loss: 1.8549e-04 - accuracy: 0.9907
Epoch 420: saving model to /home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/5 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt
2950/2950 [==============================] - 13s 4ms/step - loss: 1.8518e-04 - accuracy: 0.9907 - val_loss: 2.0625e-04 - val_accuracy: 0.9895 - lr: 8.5430e-06

Epoch 421: LearningRateScheduler setting learning rate to 7.262e-06.
Epoch 421/600
2950/2950 [==============================] - 13s 4ms/step - loss: 1.8392e-04 - accuracy: 0.9909 - val_loss: 2.0194e-04 - val_accuracy: 0.9902 - lr: 7.2620e-06

Epoch 422: LearningRateScheduler setting learning rate to 7.262e-06.
Epoch 422/600
2950/2950 [==============================] - 12s 4ms/step - loss: 1.8425e-04 - accuracy: 0.9908 - val_loss: 2.0205e-04 - val_accuracy: 0.9909 - lr: 7.2620e-06

Epoch 423

In [ ]:
lr =hist.history.get('lr')[-1] # It takes the last used learning rate.
opt = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.mean_squared_error
def ModelsWith99PercentAccuracy(epoch,lr):
    if(epoch>0):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
      return round(lr,9)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(ModelsWith99PercentAccuracy,verbose = 1)


In [ ]:
hist = model.fit(X_train,y_train,epochs = 20,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))

## Antinormalizing all the input and output features


In [ ]:
def AntiNorm(Norm_value,actual):
  return (max(actual)-min(actual))*Norm_value+min(actual)

In [ ]:
h_norm = X_norm[:,0]
v_norm = X_norm[:,1]
s_norm = X_norm[:,2]
omega_norm = X_norm[:,3]
gamma_norm = X_norm[:,4]
m_norm = X_norm[:,5]
theta_norm = X_norm[:,6]
Thrust_norm = y_norm[:,0]
beta_norm = y_norm[:,1]
h = X[:,0]
v = X[:,1]
s = X[:,2]
omega = X[:,3]
gamma = X[:,4]
m = X[:,5]
theta = X[:,6]
Thrust = y[:,0]
beta = y[:,1]

In [ ]:
# predicted outputs
y_pred = model.predict(X_norm)
Thrust_p = AntiNorm(y_pred[:,0],Thrust)
beta_p = AntiNorm(y_pred[:,1],beta)

In [ ]:
plt.plot(beta_p[0:3030])
plt.plot(beta[0:3030])
plt.legend(["Predicted","Actual"])
plt.xlabel("Number of points")
plt.ylabel("Nozzle Gamble")

plt.plot(Thrust_p[0:3030])
plt.plot(Thrust[0:3030])
plt.legend(["Predicted","Actual"])
plt.xlabel("Number of points")
plt.ylabel("Thrust")